<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/Feature%20Engineering/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
#--------------- DATA HANDLING AND MANIPULATION LIBRARIES ---------------#
import random 
import pandas as pd
import numpy as np
import os
import gc
import pickle
from scipy.stats import uniform, truncnorm, randint
from scipy.stats import randint as sp_randint
import datetime
#---------------                 Warning                  ---------------#
import warnings 
warnings.filterwarnings('ignore')

#---------------        Visualization Libraries           ---------------#
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

#---------------        Machine Learning  Libraries       ---------------#
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import RandomizedSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import xgboost as xgb

## Memory Reduction Script

In [2]:

#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Mount the drive 

In [4]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set Path

In [5]:
%cd /content/drive/MyDrive/data

/content/drive/MyDrive/data


## Load Train and Test 

In [137]:
#load train
train = reduce_mem_usage(pd.read_csv('FINAL_TRAIN_M.csv'))
test = reduce_mem_usage(pd.read_csv('FINAL_TEST_M.csv'))

Mem. usage decreased to 99.56 Mb (71.3% reduction)
Mem. usage decreased to 62.01 Mb (70.6% reduction)


## Feature Selection 

Remember not to perform any type of feature selection on Test.

### Check for null values 

In [138]:
train

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,purchasedWeekday_0_sum,purchasedWeekday_0_mean,purchasedWeekday_0_std,purchasedWeekday_1_sum,purchasedWeekday_1_mean,purchasedWeekday_1_std,purchase_Month_1_sum,purchase_Month_1_mean,purchase_Month_1_std,purchase_Month_2_sum,purchase_Month_2_mean,purchase_Month_2_std,purchase_Month_3_sum,purchase_Month_3_mean,purchase_Month_3_std,purchase_Month_4_sum,purchase_Month_4_mean,purchase_Month_4_std,purchase_Month_5_sum,purchase_Month_5_mean,purchase_Month_5_std,purchase_Month_6_sum,purchase_Month_6_mean,purchase_Month_6_std,purchase_Month_7_sum,purchase_Month_7_mean,purchase_Month_7_std,purchase_Month_8_sum,purchase_Month_8_mean,purchase_Month_8_std,purchase_Month_9_sum,purchase_Month_9_mean,purchase_Month_9_std,purchase_Month_10_sum,purchase_Month_10_mean,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff,authorized_flag_sum,authorized_flag_mean,authorized_flag_std,category_1_sum,category_1_mean,category_1_std,category_2_sum,category_2_mean,category_2_std,category_3_sum,category_3_mean,category_3_std,month_lag_sum,month_lag_mean,month_lag_min,month_lag_max,month_lag_std,installments_sum,installments_mean,installments_min,installments_max,installments_std,purchase_amount_sum,purchase_amount_mean,purchase_amount_min,purchase_amount_max,purchase_amount_std,mer_numerical_1_sum,mer_numerical_1_mean,mer_numerical_1_min,mer_numerical_1_max,mer_numerical_1_std,mer_numerical_2_sum,mer_numerical_2_mean,mer_numerical_2_min,mer_numerical_2_max,mer_numerical_2_std,mer_avg_sales_lag3_sum,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_min,mer_avg_sales_lag3_max,mer_avg_sales_lag3_std,mer_avg_sales_lag6_sum,mer_avg_sales_lag6_mean,mer_avg_sales_lag6_std,mer_avg_sales_lag12_sum,mer_avg_sales_lag12_mean,mer_avg_sales_lag12_min,mer_avg_sales_lag12_max,mer_avg_sales_lag12_std,mer_avg_purchases_lag3_sum,mer_avg_purchases_lag3_mean,mer_avg_purchases_lag3_min,mer_avg_purchases_lag3_max,mer_avg_purchases_lag3_std,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_

In [139]:
train.isna().sum()

Unnamed: 0             0
first_active_month     0
card_id                0
feature_1              0
feature_2              0
                      ..
mer_category_4_mean    0
mer_category_4_std     0
mer_category_2_sum     0
mer_category_2_mean    0
mer_category_2_std     0
Length: 225, dtype: int64

In [140]:
#if there are any inf values we will first treat it a nan
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

In [141]:
train

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,purchasedWeekday_0_sum,purchasedWeekday_0_mean,purchasedWeekday_0_std,purchasedWeekday_1_sum,purchasedWeekday_1_mean,purchasedWeekday_1_std,purchase_Month_1_sum,purchase_Month_1_mean,purchase_Month_1_std,purchase_Month_2_sum,purchase_Month_2_mean,purchase_Month_2_std,purchase_Month_3_sum,purchase_Month_3_mean,purchase_Month_3_std,purchase_Month_4_sum,purchase_Month_4_mean,purchase_Month_4_std,purchase_Month_5_sum,purchase_Month_5_mean,purchase_Month_5_std,purchase_Month_6_sum,purchase_Month_6_mean,purchase_Month_6_std,purchase_Month_7_sum,purchase_Month_7_mean,purchase_Month_7_std,purchase_Month_8_sum,purchase_Month_8_mean,purchase_Month_8_std,purchase_Month_9_sum,purchase_Month_9_mean,purchase_Month_9_std,purchase_Month_10_sum,purchase_Month_10_mean,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff,authorized_flag_sum,authorized_flag_mean,authorized_flag_std,category_1_sum,category_1_mean,category_1_std,category_2_sum,category_2_mean,category_2_std,category_3_sum,category_3_mean,category_3_std,month_lag_sum,month_lag_mean,month_lag_min,month_lag_max,month_lag_std,installments_sum,installments_mean,installments_min,installments_max,installments_std,purchase_amount_sum,purchase_amount_mean,purchase_amount_min,purchase_amount_max,purchase_amount_std,mer_numerical_1_sum,mer_numerical_1_mean,mer_numerical_1_min,mer_numerical_1_max,mer_numerical_1_std,mer_numerical_2_sum,mer_numerical_2_mean,mer_numerical_2_min,mer_numerical_2_max,mer_numerical_2_std,mer_avg_sales_lag3_sum,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_min,mer_avg_sales_lag3_max,mer_avg_sales_lag3_std,mer_avg_sales_lag6_sum,mer_avg_sales_lag6_mean,mer_avg_sales_lag6_std,mer_avg_sales_lag12_sum,mer_avg_sales_lag12_mean,mer_avg_sales_lag12_min,mer_avg_sales_lag12_max,mer_avg_sales_lag12_std,mer_avg_purchases_lag3_sum,mer_avg_purchases_lag3_mean,mer_avg_purchases_lag3_min,mer_avg_purchases_lag3_max,mer_avg_purchases_lag3_std,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_

In [142]:
# find the columns o which there are null vales 
train.columns[train.isna().any()]

Index(['mer_avg_purchases_lag3_sum', 'mer_avg_purchases_lag6_sum',
       'mer_avg_purchases_lag12_sum'],
      dtype='object')

In [143]:

def nullValues(df):
  """ Find the columns with null values """
  columns_with_na = [var for var in df.columns if df[var].isnull().mean()  > 0]
  #lets find out the percentage of observations missing per variable

  #calculate the percentage of missing 
  data_na = df[columns_with_na].isnull().mean()

  #transform the array to dataframe 
  data_na = pd.DataFrame(data_na.reset_index())

  #add names to the dataframe 
  data_na.columns = ['col','percentage_na']

  #oreder the dataframe acc to percentage 
  data_na.sort_values(by = 'percentage_na',ascending = False, inplace = True)

  #show
  data_na
  return data_na

In [144]:
print(nullValues(train))

                           col  percentage_na
2  mer_avg_purchases_lag12_sum       0.002135
1   mer_avg_purchases_lag6_sum       0.002125
0   mer_avg_purchases_lag3_sum       0.002035


In [145]:
# As the missing value percentage is very low we will mean impute the values 
train['mer_avg_purchases_lag12_sum'].fillna(train['mer_avg_purchases_lag12_sum'].mean(), inplace=True)
test['mer_avg_purchases_lag12_sum'].fillna(test['mer_avg_purchases_lag12_sum'].mean(), inplace=True)
train['mer_avg_purchases_lag6_sum'].fillna(train['mer_avg_purchases_lag6_sum'].mean(), inplace=True)
test['mer_avg_purchases_lag6_sum'].fillna(test['mer_avg_purchases_lag6_sum'].mean(), inplace=True)
train['mer_avg_purchases_lag3_sum'].fillna(train['mer_avg_purchases_lag3_sum'].mean(), inplace=True)
test['mer_avg_purchases_lag3_sum'].fillna(test['mer_avg_purchases_lag3_sum'].mean(), inplace=True)

In [146]:
print(nullValues(train))

Empty DataFrame
Columns: [col, percentage_na]
Index: []


## Basic Filter methods 

In [147]:
pd.set_option('display.max_columns', 225)
train.head()

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,purchasedWeekday_0_sum,purchasedWeekday_0_mean,purchasedWeekday_0_std,purchasedWeekday_1_sum,purchasedWeekday_1_mean,purchasedWeekday_1_std,purchase_Month_1_sum,purchase_Month_1_mean,purchase_Month_1_std,purchase_Month_2_sum,purchase_Month_2_mean,purchase_Month_2_std,purchase_Month_3_sum,purchase_Month_3_mean,purchase_Month_3_std,purchase_Month_4_sum,purchase_Month_4_mean,purchase_Month_4_std,purchase_Month_5_sum,purchase_Month_5_mean,purchase_Month_5_std,purchase_Month_6_sum,purchase_Month_6_mean,purchase_Month_6_std,purchase_Month_7_sum,purchase_Month_7_mean,purchase_Month_7_std,purchase_Month_8_sum,purchase_Month_8_mean,purchase_Month_8_std,purchase_Month_9_sum,purchase_Month_9_mean,purchase_Month_9_std,purchase_Month_10_sum,purchase_Month_10_mean,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff,authorized_flag_sum,authorized_flag_mean,authorized_flag_std,category_1_sum,category_1_mean,category_1_std,category_2_sum,category_2_mean,category_2_std,category_3_sum,category_3_mean,category_3_std,month_lag_sum,month_lag_mean,month_lag_min,month_lag_max,month_lag_std,installments_sum,installments_mean,installments_min,installments_max,installments_std,purchase_amount_sum,purchase_amount_mean,purchase_amount_min,purchase_amount_max,purchase_amount_std,mer_numerical_1_sum,mer_numerical_1_mean,mer_numerical_1_min,mer_numerical_1_max,mer_numerical_1_std,mer_numerical_2_sum,mer_numerical_2_mean,mer_numerical_2_min,mer_numerical_2_max,mer_numerical_2_std,mer_avg_sales_lag3_sum,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_min,mer_avg_sales_lag3_max,mer_avg_sales_lag3_std,mer_avg_sales_lag6_sum,mer_avg_sales_lag6_mean,mer_avg_sales_lag6_std,mer_avg_sales_lag12_sum,mer_avg_sales_lag12_mean,mer_avg_sales_lag12_min,mer_avg_sales_lag12_max,mer_avg_sales_lag12_std,mer_avg_purchases_lag3_sum,mer_avg_purchases_lag3_mean,mer_avg_purchases_lag3_min,mer_avg_purchases_lag3_max,mer_avg_purchases_lag3_std,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_

In [148]:
#drop the unnamed 0
cols = 'Unnamed: 0'
train.drop(cols, axis = 1, inplace = True)

In [149]:
# ignore id based columns and date based columns 
columnsToTake = []
for col in train.columns:
    if col != 'card_id' and col != 'first_active_month' and col != 'target':
        columnsToTake.append(col)

In [150]:
len(columnsToTake)

221

In [151]:
sel = VarianceThreshold(threshold = 0)
sel.fit(train[columnsToTake])

VarianceThreshold(threshold=0)

In [152]:
# get support is a boolean vector that indicates which features are retained 
# if we sum over get_support, we get the umber of features that are not constant
sum(sel.get_support())


219

In [153]:
# Two constant columns 
temp = train[columnsToTake]
[x for x in temp.columns if x not in temp.columns[sel.get_support()]]

['mer_active_months_lag3_max', 'mer_active_months_lag6_max']

In [154]:
temp.mer_active_months_lag3_max.value_counts()/np.float(len(temp))

3.0    1.0
Name: mer_active_months_lag3_max, dtype: float64

In [155]:
temp.mer_active_months_lag6_max.value_counts()/np.float(len(temp))

6.0    1.0
Name: mer_active_months_lag6_max, dtype: float64

In [156]:
def remove_noninformative(df):
    """Remove non-informative columns (all nan, or all same value)"""
    for col in df:
        if df[col].isnull().all():
            print('Removing '+col+' (all NaN)')
            del df[col]
        elif df[col].nunique()<2:
            print('Removing '+col+' (only 1 unique value)')
            del df[col]

remove_noninformative(temp)
gc.collect()

Removing mer_active_months_lag3_max (only 1 unique value)
Removing mer_active_months_lag6_max (only 1 unique value)


202

In [157]:
for col in columnsToTake:
    if col is "mer_active_months_lag3_max" or col is "mer_active_months_lag6_max":
        columnsToTake.remove(col)

In [158]:
len(columnsToTake)

221

In [127]:
quaziConstantFeatures = []
for feat in temp.columns:
    pre = (temp[feat].value_counts()/np.float(
    len(temp))).sort_values(ascending = False).values[0]
    
    if pre > 0.99:
        quaziConstantFeatures.append(feat)
len(quaziConstantFeatures)

3

In [128]:
quaziConstantFeatures

['mer_numerical_1_min', 'mer_numerical_2_min', 'mer_active_months_lag12_max']

In [129]:
temp.mer_numerical_1_min.value_counts()/np.float(len(temp))

-0.057465     0.991967
-0.047546     0.003853
-0.037628     0.001332
-0.027725     0.000797
-0.017807     0.000287
                ...   
 0.220093     0.000005
 20.500000    0.000005
 0.755371     0.000005
 7.152344     0.000005
 1.003906     0.000005
Name: mer_numerical_1_min, Length: 106, dtype: float64

In [130]:
temp.mer_numerical_2_min.value_counts()/np.float(len(temp))

-0.057465    0.997519
-0.047546    0.001030
-0.027725    0.000426
-0.037628    0.000406
-0.017807    0.000074
               ...   
 0.160645    0.000005
 0.755371    0.000005
 0.448242    0.000005
 2.599609    0.000005
 6.894531    0.000005
Name: mer_numerical_2_min, Length: 83, dtype: float64

In [131]:
temp.mer_active_months_lag12_max.value_counts()/np.float(len(temp))

12.0    0.999802
10.0    0.000183
11.0    0.000005
8.0     0.000005
9.0     0.000005
Name: mer_active_months_lag12_max, dtype: float64

In [133]:
columnsToTake = temp.columns

In [134]:
len(columnsToTake)

219

In [135]:
for col in columnsToTake:
    if col in quaziConstantFeatures:
        columnsToTake.remove(col)

AttributeError: ignored

In [136]:
len(columnsToTake)

219

In [117]:
# for duplicate features 
duplicatedFeat = []
for i in range(0, len(columnsToTake)):
    
    if i % 10 == 0:
        print(i)
        
    col1 = temp.columns[i]
    
    for col2 in temp.columns[i + 1:]:
        if temp[col1].equals(temp[col2]):
            print(col1)
            print(col2)
            duplicatedFeat.append(col2)

0
feature_3
feature_min
10
20
30
40
50
purchasedWeekday_0_sum
purchasedWeekend_1_sum
purchasedWeekday_0_mean
purchasedWeekend_1_mean
purchasedWeekday_0_std
purchasedWeekday_1_std
purchasedWeekday_0_std
purchasedWeekend_0_std
purchasedWeekday_0_std
purchasedWeekend_1_std
purchasedWeekday_1_sum
purchasedWeekend_0_sum
purchasedWeekday_1_mean
purchasedWeekend_0_mean
purchasedWeekday_1_std
purchasedWeekend_0_std
purchasedWeekday_1_std
purchasedWeekend_1_std
60
70
80
90
purchasedWeekend_0_std
purchasedWeekend_1_std
100
110
120
130
140
150
160
170
180
190
200
210


In [118]:
duplicatedFeat

['feature_min',
 'purchasedWeekend_1_sum',
 'purchasedWeekend_1_mean',
 'purchasedWeekday_1_std',
 'purchasedWeekend_0_std',
 'purchasedWeekend_1_std',
 'purchasedWeekend_0_sum',
 'purchasedWeekend_0_mean',
 'purchasedWeekend_0_std',
 'purchasedWeekend_1_std',
 'purchasedWeekend_1_std']

In [119]:
for col in columnsToTake:
    if col in duplicatedFeat:
        columnsToTake.remove(col)

In [120]:
len(columnsToTake)

213

In [122]:
for col in columnsToTake:
    if col is "mer_active_months_lag3_max" or col is "mer_active_months_lag6_max":
        columnsToTake.remove(col)

In [123]:
len(columnsToTake)

213

## Correlation 

In [124]:
temp = temp[columnsToTake]
temp

KeyError: ignored

In [ ]:
corr = temp.corr()
fig, ax = plt.subplots()
fig.set_size_inches(11,11)
sns.heatmap(corr, vmin=-1, vmax=1, cmap='BrBG')